In [13]:
from pandas_geojson import read_geojson
from pyproj import Transformer, Geod
import json

In [5]:

f = './GeoJSON_Exports/USGS_Wildland_Fire_Combined_Dataset.json'
# geo_json = read_geojson(f)

In [6]:
import geojson
with open(f) as f:
    gj = geojson.load(f)
features = gj['features'][0]


In [23]:
features = gj['features'][0:]

print(features[4]['attributes']['Fire_Year'])
print(features[1]['attributes'])
print(type(features[1]['attributes']['Fire_Year']))


1870
{'OBJECTID': 2, 'USGS_Assigned_ID': 2, 'Assigned_Fire_Type': 'Wildfire', 'Fire_Year': 1860, 'Fire_Polygon_Tier': 1, 'Fire_Attribute_Tiers': '1 (1)', 'GIS_Acres': 772.5182488669103, 'GIS_Hectares': 312.62704368482576, 'Source_Datasets': 'Comb_National_NIFC_Interagency_Fire_Perimeter_History (1)', 'Listed_Fire_Types': 'Wildfire (1)', 'Listed_Fire_Names': 'Harrison Lake (1)', 'Listed_Fire_Codes': 'No code provided (1)', 'Listed_Fire_IDs': '', 'Listed_Fire_IRWIN_IDs': '', 'Listed_Fire_Dates': 'Listed Other Fire Date(s): 2006-11-02 - NIFC DATE_CUR field (1)', 'Listed_Fire_Causes': '', 'Listed_Fire_Cause_Class': 'Undetermined (1)', 'Listed_Rx_Reported_Acres': None, 'Listed_Map_Digitize_Methods': 'Other (1)', 'Listed_Notes': '', 'Processing_Notes': '', 'Wildfire_Notice': 'Wildfire mapping prior to 1984 was inconsistent, infrequent, and done without the aid of more modern fire mapping methods (GPS and satellite imagery). Areas burned prior to 1984 in this dataset represent only a fraction

In [39]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    threshold_distance = 1250
    # run through each point in the converted ring data
    for point in ring:
        
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # print(distance_in_miles)
        if distance_in_miles > threshold_distance + 500:
            return [2*threshold_distance]
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point

In [12]:
print(len(features))
# https://ww2.arb.ca.gov/resources/inhalable-particulate-matter-and-health

135061


In [42]:
CITY_LOCATIONS = {
    'pullman' :       {'city'   : 'Pullman',
                       'county' : 'Whitman',
                       'state'  : 'Washington',
                       'fips'   : '53075',
                       'latlon' : [46.7298, -117.1817] }, 
   }
place = CITY_LOCATIONS['pullman']['latlon']
ring = features[1000]['geometry']['rings'][0]
threshold_distance = 1250
# 1389.760591262309
# fires = []
for i in range(34353,len(features)):
    try:
        if i%1000 == 0:
            print(i)
        if features[i]['attributes']['Fire_Year'] > 1963:
            ring = features[i]['geometry']['rings'][0]
            if shortest_distance_from_place_to_fire_perimeter(place, ring)[0] < threshold_distance:
                fires.append(features[i])
    except:
         print('failed')
with open('fires2.json', 'w') as file:
            json.dump(fires, file)

35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
failed
110000
failed
failed
111000
failed
failed
failed
112000
failed
failed
113000
failed
failed
failed
failed
failed
114000
failed
failed
115000
failed
failed
116000
failed
117000
failed
118000
119000
failed
failed
failed
failed
120000
failed
failed
failed
failed
121000
failed
122000
failed
failed
123000
failed
124000
failed
125000
failed
failed
126000
127000
failed
128000
129000
130000
131000
132000
133000
134000
135000


In [ ]:
fires = []
CITY_LOCATIONS = {
    'pullman' :       {'city'   : 'Pullman',
                       'county' : 'Whitman',
                       'state'  : 'Washington',
                       'fips'   : '53075',
                       'latlon' : [46.7298, -117.1817] }, 
   }
place = CITY_LOCATIONS['pullman']['latlon']
for i in range(0,len(gj)):
    try:
        if i%1000 == 0:
            print(i)
        
        ring = gj[i]['geometry']['rings'][0]

        selected_keys = ['USGS_Assigned_ID', 'Fire_Year', 'Assigned_Fire_Type', 'GIS_Acres', 'GIS_Hectares']
        info ={key: gj[i]['attributes'][key] for key in selected_keys}
        info['Distance'] = shortest_distance_from_place_to_fire_perimeter(place, ring)[0]
        fires.append(info)
    except:
         print('failed')
    
with open('fires.json', 'w') as file:
            json.dump(fires, file)

34353
